In [1]:
import pandas as pd
path_train = "/Users/gunnvantsaini/Documents/Data/Kaggle/News/BBC News Train.csv"
train = pd.read_csv(path_train)

In [2]:
train.head(2)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business


In [3]:
## We will create a classifier using embedding layer architecture
X=train['Text']
y=train['Category']

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=200)

In [6]:
enc=LabelEncoder()

In [7]:
y_train=enc.fit_transform(y_train)

In [8]:
enc.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [9]:
y_train

array([4, 3, 0, ..., 3, 1, 1])

In [10]:
import tensorflow as tf

In [11]:
#### tf2.2+ ####
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [12]:
seq_len=16

In [13]:
vectorizer=TextVectorization(max_tokens=30000,output_sequence_length=seq_len) ### tfidf object

Metal device set to: Apple M1


2022-02-01 10:33:11.562601: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-01 10:33:11.563517: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
text_ds=tf.data.Dataset.from_tensor_slices(X_train).batch(64)

In [15]:
vectorizer.adapt(text_ds) ### fit_transform

2022-02-01 10:33:29.076923: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-01 10:33:29.077698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [16]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [17]:
len(vectorizer.get_vocabulary())

24865

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

In [19]:
inputs = keras.Input(shape=(1,),dtype='string')
x = vectorizer(inputs)
x = layers.Embedding(input_dim=24865,output_dim=64)(x) ### [obj1,obj2,...obj16] ### 50, 100,300
x = layers.LSTM(100)(x)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dense(4,activation='softmax')(x)

In [20]:
model=tf.keras.Model(inputs,x)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 16)                0         
_________________________________________________________________
embedding (Embedding)        (None, 16, 64)            1591360   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 1,664,084
Trainable params: 1,664,084
Non-trainable params: 0
___________________________________________________

In [22]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])

In [23]:
model.fit(X_train,y_train,batch_size=128,epochs=5,validation_split=0.10)

Epoch 1/5


2022-02-01 10:39:34.205643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-01 10:39:34.386718: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-01 10:39:35.026499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 3s 71ms/step - loss: 1.1363 - acc: 0.2229 - val_loss: 1.2222 - val_acc: 0.2583
Epoch 2/5
1/9 [==>...........................] - ETA: 0s - loss: 1.1410 - acc: 0.2812

2022-02-01 10:39:36.433157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-01 10:39:36.489979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 29ms/step - loss: 1.1574 - acc: 0.2220 - val_loss: 1.2591 - val_acc: 0.2917
Epoch 3/5
9/9 [==============================] - 0s 29ms/step - loss: 1.1506 - acc: 0.1978 - val_loss: 1.2861 - val_acc: 0.2583
Epoch 4/5
9/9 [==============================] - 0s 30ms/step - loss: 1.1506 - acc: 0.2127 - val_loss: 1.3159 - val_acc: 0.1250
Epoch 5/5
9/9 [==============================] - 0s 29ms/step - loss: 1.1463 - acc: 0.2062 - val_loss: 1.2787 - val_acc: 0.2583
